# Image Preprocessing

Preprocess the image to isolate the iris from the rest of the image. This involves several steps, including:

Localization: Identify the iris within the image.
Normalization: Adjust the image to a common scale and orientation.
Enhancement: Improve the image quality, often by increasing contrast or reducing noise.

In [1]:
import cv2
import pywt
import numpy as np
from scipy.interpolate import RectBivariateSpline
from shapely.geometry import Point, Polygon
from skimage import filters
import numpy as np
import hashlib

# Iris Localization
def iris_localization(image):
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # apply Gaussian blur
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150)

    # apply Hough Circle Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

    # find the circle with the largest radius (assuming this is the iris)
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        # sort the circles by radius in descending order
        circles = sorted(circles, key=lambda x: -x[2])
        x, y, r = circles[0]
        iris_region = image[y-r:y+r, x-r:x+r]
    else:
        iris_region = None

    return iris_region

def iris_normalization(iris_region, pupil_center, iris_radius, eyelid_contour=None):
    # ensure iris_region is grayscale
    if len(iris_region.shape) == 3:
        iris_region = cv2.cvtColor(iris_region, cv2.COLOR_BGR2GRAY)

    # define the dimensions of the normalized iris image
    rows, cols = (32, 256)
    normalized_iris = np.zeros((rows, cols))

    # create an interpolation function for the iris region
    iris_region_interpolate = RectBivariateSpline(np.arange(iris_region.shape[0]), np.arange(iris_region.shape[1]), iris_region)

    # create a polygon for the eyelid contour if one is provided
    if eyelid_contour is not None:
        eyelid_polygon = Polygon(eyelid_contour)

    # compute the angle and radius for each point in the normalized iris image
    for i in range(rows):
        for j in range(cols):
            theta = 2*np.pi*j/cols
            r = i/rows
            x = pupil_center[0] + r*iris_radius*np.cos(theta)
            y = pupil_center[1] + r*iris_radius*np.sin(theta)

            # check if the point is within the eyelid contour, if one is provided
            if eyelid_contour is not None and eyelid_polygon.contains(Point(x, y)):
                # if the point is within the eyelid contour, assign a default value
                normalized_iris[i, j] = 0  # or any other default value
            else:
                # check if the point falls inside the iris region
                if 0 <= x < iris_region.shape[1] and 0 <= y < iris_region.shape[0]:
                    normalized_iris[i, j] = iris_region_interpolate(y, x)[0, 0]  # extract the scalar value
                else:
                    # if the point falls outside the iris region, assign a default value
                    normalized_iris[i, j] = 0  # or any other default value

    return normalized_iris

# Feature Extraction
Extract features from the image. These features represent the unique aspects of the individual's iris. Various methods can be used for feature extraction, including wavelet transforms, Gabor filters, and others.

In [3]:
def gabor_feature_extraction(normalized_iris):
    # define the orientations and frequencies for the Gabor filters
    orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    frequencies = [0.2, 0.4, 0.6, 0.8]
    gabor_features = []

    for orientation in orientations:
        for frequency in frequencies:
            gabor_response_real, gabor_response_imag = filters.gabor(normalized_iris, frequency=frequency, theta=orientation)
            gabor_features.extend(gabor_response_real.flatten())
            gabor_features.extend(gabor_response_imag.flatten())

    return np.array(gabor_features)


def wavelet_feature_extraction(normalized_iris):
    # perform a wavelet transform on the normalized iris
    coeffs = pywt.wavedec2(normalized_iris, 'haar')

    # flatten the wavelet coefficients to create a feature vector
    wavelet_features = np.hstack([coeff.ravel() for coeff in coeffs[0]] +
                             [value.ravel() for coeff in coeffs[1:] for value in coeff])

    return wavelet_features

def feature_extraction(normalized_iris):
    # apply Gabor filters for feature extraction
    gabor_features = gabor_feature_extraction(normalized_iris)

    # apply wavelet transform for feature extraction
    wavelet_features = wavelet_feature_extraction(normalized_iris)

    # concatenate the Gabor and wavelet features to create a combined feature vector
    features = np.concatenate([gabor_features, wavelet_features])

    return features

# Post-processing

Use the result of the matching process to make a decision. This might involve accepting or rejecting the individual's claimed identity, or it could involve further steps such as updating the database or notifying security personnel.

Matching: The extracted features are then compared to the features in a database of known individuals. If the features match closely enough, the individual is identified.

Post-processing: Finally, the result of the matching process is used to make a decision. This might involve accepting or rejecting the individual's claimed identity, or it could involve further steps such as updating the database or notifying security personnel.

In [4]:
# Example usage
image = cv2.imread("iris_image/2/left/bryanl1.bmp")
iris_region = iris_localization(image)

if iris_region is not None:
    # placeholder values for pupil_center, iris_radius, and eyelid_contour
    pupil_center = (iris_region.shape[1]//2, iris_region.shape[0]//2)
    iris_radius = min(pupil_center)
    eyelid_contour = None  # no eyelid contour is provided

    normalized_iris = iris_normalization(iris_region, pupil_center, iris_radius, eyelid_contour)
    features = feature_extraction(normalized_iris)
    print(features)

[ 0.03978455  0.0362278   0.02970633 ... -2.71562533 -1.51401062
 -3.2516927 ]


In [7]:
# Convert the array to a byte string
array_string = features.tobytes()

# Create a SHA-256 hash of the byte string
array_hash = hashlib.sha256(array_string).hexdigest()

print("Array hash:", array_hash.upper())

Array hash: 39FD3D3302FBC44D985451E849B65DC4F9EB58AB1242125C86AF4235008ECB6B
